In [1]:
import pandas as pd
import numpy as np
from keras.src.optimizers import Adam
from skimage import io
from skimage.transform import resize
from skimage.feature import hog
from PIL import Image
import os
import shutil
import glob
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
import zipfile
import os

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
zip_path ='/content/drive/MyDrive/thermalCamera/Mixture.zip'


In [5]:
extraction_path ='/content/thermalCamera/Mixture'

In [6]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [7]:
zip_path ='/content/drive/MyDrive/thermalCamera/Smoke.zip'
extraction_path ='/content/thermalCamera/Smoke'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [8]:
zip_path ='/content/drive/MyDrive/thermalCamera/Perfume.zip'
extraction_path ='/content/thermalCamera/Perfume'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [9]:
zip_path ='/content/drive/MyDrive/thermalCamera/NoGas.zip'
extraction_path ='/content/thermalCamera/NoGas'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [ ]:
os.listdir(extraction_path)

In [11]:
import shutil
def split_data(source_folder, train_folder, val_folder, test_folder, train_size=0.7, val_size=0.15):
    classes = os.listdir(source_folder)
    for cls in classes:
        class_dir = os.path.join(source_folder, cls)
        if not os.path.isdir(class_dir):  # Skip if not a directory
            continue

        os.makedirs(os.path.join(train_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(val_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(test_folder, cls), exist_ok=True)

        images = [img for img in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, img))]
        np.random.shuffle(images)
        train_index = int(train_size * len(images))
        val_index = train_index + int(val_size * len(images))

        train_images = images[:train_index]
        val_images = images[train_index:val_index]
        test_images = images[val_index:]

        for img in train_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(train_folder, cls, img))
        for img in val_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(val_folder, cls, img))
        for img in test_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(test_folder, cls, img))
split_data('/content/thermalCamera', '/content/thermalCamera/train_data', '/content/thermalCamera/validation_data', '/content/thermalCamera/test_data', train_size=0.64, val_size=0.16)

In [ ]:
def count_images(folder_path):
    for root, dirs, files in os.walk(folder_path):
        # Filter out directories, count only files
        file_count = len([file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))])
        print(f"{root} contains {file_count} image(s)")

main_directory = '/content/thermalCamera'
count_images(main_directory)

In [13]:
from keras.preprocessing.image import ImageDataGenerator
from torchvision import datasets
from torch.utils.data import DataLoader

train_datagen = ImageDataGenerator(
    rescale=1./255,
    # Additional augmentation parameters can be added here
)
train_generator = train_datagen.flow_from_directory(
    '/content/thermalCamera/train_data',
    target_size=(400, 600),
    batch_size=32,
    class_mode='categorical',  # Change as per your problem
    color_mode='rgb'
)

test_generator = train_datagen.flow_from_directory(
    '/content/thermalCamera/test_data',
    target_size=(400, 600),
    batch_size=32,
    class_mode='categorical',  # Change as per your problem
    color_mode='rgb'
)
validation_generator = train_datagen.flow_from_directory(
    '/content/thermalCamera/validation_data',
    target_size=(400, 600),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)
dataset = datasets.ImageFolder('/content/thermalCamera')
val_dataset = datasets.ImageFolder('/content/thermalCamera/validation_data')
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

Found 4096 images belonging to 4 classes.
Found 1280 images belonging to 4 classes.
Found 1024 images belonging to 4 classes.


In [16]:
model = Sequential()

# Convolution-Pooling Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(400, 600, 3), kernel_regularizer=l1_l2(l1=0.005, l2=0.005)))
model.add(MaxPooling2D((2, 2)))

# Convolution-Pooling Layer 2 with L1-L2 regularization
model.add(Conv2D(25, (3, 3), activation='relu', kernel_regularizer=l1_l2(l1=0.005, l2=0.005)))
model.add(MaxPooling2D((2, 2)))

# Convolution-Pooling Layer 3
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten layer
model.add(Flatten())

# Dropout layer
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

optimizer = Adam(learning_rate=0.001, decay=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=32 // train_generator.batch_size,
    epochs=250,
    validation_data=validation_generator,
    validation_steps=32 // validation_generator.batch_size
)

Epoch 1/250
1/1 [==============================] - 22s 22s/step - loss: 3.7862 - accuracy: 0.2812 - val_loss: 3.7273 - val_accuracy: 0.2812
Epoch 2/250
1/1 [==============================] - 17s 17s/step - loss: 3.6786 - accuracy: 0.3438 - val_loss: 3.5986 - val_accuracy: 0.5312
Epoch 3/250
1/1 [==============================] - 18s 18s/step - loss: 3.5731 - accuracy: 0.4688 - val_loss: 3.4061 - val_accuracy: 0.5312
Epoch 4/250
1/1 [==============================] - 17s 17s/step - loss: 3.3997 - accuracy: 0.5938 - val_loss: 3.1653 - val_accuracy: 0.6250
Epoch 5/250
1/1 [==============================] - 17s 17s/step - loss: 3.1305 - accuracy: 0.6875 - val_loss: 2.9966 - val_accuracy: 0.6875
Epoch 6/250
1/1 [==============================] - 22s 22s/step - loss: 2.9850 - accuracy: 0.7500 - val_loss: 2.9641 - val_accuracy: 0.5938
Epoch 7/250
1/1 [==============================] - 16s 16s/step - loss: 2.9312 - accuracy: 0.6875 - val_loss: 3.0181 - val_accuracy: 0.6250
Epoch 8/250
1/1 [===

In [17]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test accuracy: ", test_accuracy)

40/40 [==============================] - 145s 4s/step - loss: 0.8814 - accuracy: 0.9211
Test accuracy:  0.921093761920929
